In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
from animal_Shelter import AnimalShelter  # Ensure this import matches your file
import plotly.express as px

# MongoDB Connection
username = "aacuser"
password = "Alice2018"
animals = AnimalShelter(username, password)

# Retrieve data from MongoDB
df = pd.DataFrame.from_records(animals.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Encode the Grazioso Salvare logo for display
image_filename = '/home/rafaelcanseco_snhu/Desktop/Grazioso Salvare Logo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Initialize the Dash app
app = JupyterDash(__name__)

# Layout of the Dash app
app.layout = html.Div(style={'padding': '20px'}, children=[
    html.Center(html.H1('CS-340 Dashboard - Rafael Canseco', style={'margin-bottom': '20px'})),
    
    # Center the logo with some margin
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px', 'margin-bottom': '20px'})),
    
    # Radio buttons for filtering Rescue Types
    html.Div(style={'text-align': 'center', 'margin-bottom': '20px'}, children=[
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}  
            ],
            value='Reset',  # Default filter
            labelStyle={'display': 'inline-block', 'padding': '10px'}
        )
    ]),
    
    # DataTable with less clutter and improved spacing
    html.Div(style={'overflowX': 'auto', 'margin-bottom': '20px'}, children=[
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i.replace("_", " ").capitalize(), "id": i} for i in df.columns],
            data=df.to_dict('records'),
            editable=False,
            filter_action="native",
            sort_action="native",
            row_selectable="single",
            page_size=8,  # Show fewer rows to avoid clutter
            style_table={'maxHeight': '300px', 'overflowY': 'auto'},  # Limit table height
            style_cell={'padding': '10px', 'textAlign': 'left'},
            style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold', 'textAlign': 'center'},
            style_data={'border': '1px solid grey'}
        )
    ]),
    
    # Row for map and chart
    html.Div(style={'display': 'flex', 'justify-content': 'space-between', 'gap': '20px'}, children=[
        html.Div(id='graph-id', style={'flex': '1'}),
        html.Div(id='map-id', style={'flex': '1'})
    ])
])

# Callback to filter data table based on the selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        filtered_data = pd.DataFrame.from_records(animals.read({}))
    else:
        filtered_data = pd.DataFrame.from_records(animals.read({'rescue_type': filter_type}))
    
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)
    
    return filtered_data.to_dict('records')

# Callback to update the pie chart based on the filtered data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        return "No data available"
    
    dff = pd.DataFrame(viewData)
    if dff.empty or 'outcome_type' not in dff.columns:
        return "No outcome data available"
    
    # Pie chart for outcome_type distribution
    fig = px.pie(dff, names='outcome_type', title='Outcome Type Distribution')
    return dcc.Graph(figure=fig)

# Callback to update the map based on selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData) == 0 or selected_rows is None:
        return dl.Map(style={'width': '100%', 'height': '400px'}, center=[30.2672, -97.7431], zoom=10, children=[dl.TileLayer()])
    
    dff = pd.DataFrame.from_dict(viewData)
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return "No location data available"
    
    row = selected_rows[0] if selected_rows else 0
    return dl.Map(
        style={'width': '100%', 'height': '400px'},
        center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10,
        children=[
            dl.TileLayer(),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['name'] if dff.iloc[row]['name'] else 'No Name'),
                dl.Popup([
                    html.H4(f"Animal: {dff.iloc[row]['name']}" if dff.iloc[row]['name'] else 'No Name'),
                    html.P(f"Breed: {dff.iloc[row]['breed']}"),
                    html.P(f"Outcome: {dff.iloc[row]['outcome_type']}")
                ])
            ])
        ]
    )

# Run the Dash app
app.run_server(debug=True, port=31162, host='127.0.0.2')







Dash app running on http://127.0.0.2:31162/
